In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import cv2 
import tensorflow as tf
from matplotlib import pyplot as plt
import numpy as np
import os

In [3]:
base__train_dir = "../input/lung-disease-classification/train_/train_"
base_test_dir = "../input/lung-disease-classification/test_/test_"

In [4]:
image_length_size = 300
image_height_size = 200
batch_size = 32

In [5]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 40,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip  = True,
    fill_mode='nearest',
    validation_split = 0.1
)

In [6]:
train_generator = train_datagen.flow_from_directory(
    base__train_dir,
    target_size = (image_length_size,image_height_size),
    batch_size = batch_size,
    class_mode='sparse',
    subset = 'training',
    shuffle = True
)

Found 10922 images belonging to 5 classes.


In [7]:
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 40,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip =True,
    fill_mode = 'nearest',
    validation_split = 0.1
)

In [8]:
test_generator = test_datagen.flow_from_directory(
    base_test_dir,
    target_size = (image_length_size,image_height_size),
    batch_size = batch_size,
    class_mode='sparse',
    subset = 'validation',
    shuffle=True
)

Found 137 images belonging to 5 classes.


In [9]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(image_length_size, image_height_size, 3)),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(64, (3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(32, (3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    #tf.keras.layers.Conv2D(64, (3,3), activation='relu'), 
    #tf.keras.layers.MaxPooling2D(2,2),
    ## Flatten the results to feed into a DNN
    #tf.keras.layers.Flatten(), 
    ## 512 neuron hidden layer
    #tf.keras.layers.Dense(512, activation='relu'), 
    ## Only 48 output neuron.
    tf.keras.layers.Dense(5, activation='softmax')  
])

2023-01-28 05:33:53.548553: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-28 05:33:53.642237: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-28 05:33:53.643022: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-28 05:33:53.645456: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 298, 198, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 149, 99, 32)       0         
_________________________________________________________________
dropout (Dropout)            (None, 149, 99, 32)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 147, 97, 64)       18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 48, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 73, 48, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 46, 128)       7

In [11]:
model.compile(optimizer= tf.keras.optimizers.RMSprop(learning_rate=1e-4),
              loss='sparse_categorical_crossentropy',
              metrics = ['accuracy'])

In [12]:
history = model.fit(
            train_generator,
            epochs=20,
            steps_per_epoch = 300,
    # 92k images = batch_size*steps
    #i.e steps = 10k/32 = 312
            #steps_per_epoch=total_train_batches//hvd.size(),
            validation_data=test_generator,
            verbose=1
            )

2023-01-28 05:33:57.185559: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/20


2023-01-28 05:33:59.607399: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


300/300 [==============================] - 189s 603ms/step - loss: 1.2525 - accuracy: 0.5119 - val_loss: 1.5340 - val_accuracy: 0.3942
Epoch 2/20
300/300 [==============================] - 143s 477ms/step - loss: 1.1093 - accuracy: 0.5830 - val_loss: 1.5186 - val_accuracy: 0.4234
Epoch 3/20
300/300 [==============================] - 142s 472ms/step - loss: 0.9789 - accuracy: 0.6304 - val_loss: 1.4255 - val_accuracy: 0.5328
Epoch 4/20
300/300 [==============================] - 139s 464ms/step - loss: 0.8859 - accuracy: 0.6714 - val_loss: 1.3357 - val_accuracy: 0.5693
Epoch 5/20
300/300 [==============================] - 137s 457ms/step - loss: 0.8317 - accuracy: 0.6910 - val_loss: 1.2595 - val_accuracy: 0.6277
Epoch 6/20
300/300 [==============================] - 138s 461ms/step - loss: 0.7965 - accuracy: 0.7072 - val_loss: 1.2008 - val_accuracy: 0.6423
Epoch 7/20
300/300 [==============================] - 138s 458ms/step - loss: 0.7677 - accuracy: 0.7177 - val_loss: 1.1537 - val_accura

In [13]:
# #Save Model
# saved_model = 'Haude_boiz.sav'
# pickle.dump(model, open(saved_model, 'wb'))

In [14]:
# #Load Model 
# loaded_model = pickle.load(open(saved_model, 'rb'))
# result = loaded_model.score(X_test, Y_test)
# print(result)

In [15]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")

In [16]:
from keras.models import model_from_json

# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")